# <font color='darkblue'>2. Q Learning Framework</font>

## <font color='darkblue'>Copyright 2019 Google LLC.</font>

In [1]:
#@title
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This Colab is part of the [Reinforcement Learning course](https://developers.google.com/machine-learning/reinforcement-learning/). In the previous Colab, you learned to frame problems in reinforcement learning. In this Colab, you will learn about the RL decision-making process by applying the following concepts:

* [Markov Decision Process](https://en.wikipedia.org/wiki/Markov_decision_process) (MDP)
* Expected return and Q-value
* Bellman equation

Lastly, you will use these concepts to solve the `NChain-v0` environment.

<a id='sect0'></a>
### <font color='darkgreen'>Agenda</font>
* <font size='3ptx'>[**Markov Decision Process**](#sect1)</font>
* <font size='3ptx'>[**Learning Rewards**](#sect2)</font>
    * [**Solution (expand to view)**](#sect2_1)
* <font size='3ptx'>[**Learning Probabilistic Rewards**](#sect3)</font>
    * [**Solution (expand to view code)**](#sect3_1)
* <font size='3ptx'>[**Q-Function and Q-Values**](#sect4)</font>
* <font size='3ptx'>[**Implement Bellman Equation**](#sect5)</font>
* <font size='3ptx'>[**Train the Agent to Solve NChain**](#sect6)</font>
    * [**Answers**](#sect6_1)
    * [**Run Training**](#sect6_2)
    * [**Solution**](#sect6_3)
* <font size='3ptx'>[**Test Your Trained Agent**](#sect7)</font>
* <font size='3ptx'>[**Contrasting RL with Supervised Learning**](#sect8)</font>
* <font size='3ptx'>[**Conclusion and Next Steps**](#sect9)</font>

## <font color='darkblue'>Setup</font>

Run the following cell to setup Google Analytics for the Colab. Data from  Google Analytics helps improve the Colab.

In [2]:
#@title Set up Google Analytics for Colab
%reset -f
import uuid
client_id = uuid.uuid4()

import requests

# Bundle up reporting into a function.
def report_execution():
  requests.post('https://www.google-analytics.com/collect', 
                data=('v=1'
                      '&tid=UA-48865479-3'
                      '&cid={}'
                      '&t=event'
                      '&ec=cell'            # <-- event type
                      '&ea=execute'         # <-- event action
                      '&el=rl-q-learning'   # <-- event label
                      '&ev=1'               # <-- event value
                      '&an=bundled'.format(client_id)))

from IPython import get_ipython
get_ipython().events.register('post_execute', report_execution)

Run the following cell to import libraries:

In [3]:
import gym
import numpy as np
from IPython.display import clear_output

np.set_printoptions(precision=2, suppress=True)

<a id='sect1'></a>
## <font color='darkblue'>Markov Decision Process</font> ([back](#sect0))

In the previous Colab, you explored the `NChain-v0` environment, visualized in the following graph:

<img alt="A schematic that shows the NChain environment. The schematic shows the states, possible actions, and results of taking those actions in the state. When an agent takes an action in a state, the agent moves to a new state and receives a reward. There are 5 states. The allowed actions from each state are labelled 0 and 1. Action 0 always leads to a reward of 0, except from state 4 where action 0 returns a reward of 10. Action 1 always returns a reward of 2." width="75%" src="images/2.PNG"/>

Suppose you're in state 4. You know that action 0 returns a big reward with high probability.  **This probability only depends on the current state, 4, and not on the previous sequence of states. This property, where possible state transitions are completely determined by the current state, is called the <font color='darkblue'>Markov property</font>**.

From state 4, you decide to take action 0 to get the big reward. To make your decision, again, you only needed to know your current state. You didn't need knowledge of *how* you reached state 4. **When an agent makes decisions to navigate a sequence of states under the Markov property, then the result is called a <font color='darkblue'>Markov Decision Process</font> (MDP)**.

Recall that this sequence of states is called a <font color='darkblue'>**trajectory**</font>, represented by $(s,a,r,s')$ tuples as follows:

$$s_0 \xrightarrow[r_0]{a_0} s_1 \xrightarrow[r_1]{a_1} s_2 \ldots \xrightarrow[r_2]{a_{n-2}} s_{n-1}\xrightarrow[r_{n-1}]{a_{n-1}} s_n
$$

<a id='sect2'></a>
## <font color='darkblue'>Learning Rewards</font> ([back](#sect0))

Under the Markov property, you know that the state-action pair s=4, a=0 will  probably return r=10, s'=4. That is, the next state and associated reward depend solely on the current state.

<b>Since rewards are specific to state-action pairs, you can track rewards for each state-action pair</b>. First, for the `NChain-v0` environment, get the number of states and actions:

In [4]:
env = gym.make("NChain-v0")
state = env.reset()

num_states = env.observation_space.n
num_actions = env.action_space.n

print("NChain-v0 has " + str(num_states) + " states and " + str(num_actions) + " actions.")

NChain-v0 has 5 states and 2 actions.


Create a table to track rewards for each state-action transition. By default, initialize rewards to `0`.

This table stores <font color='darkblue'>**Q-values**</font> for every state-action pair in the following format:

$$
\text{states}\left\downarrow\vphantom{
% the following matrix is invisible
% the \vphantom uses the matrix to know its size
\begin{bmatrix} 
Q(s=0,a=0) & Q(s=0,a=1) \\
Q(s=1,a=0) & Q(s=1,a=1) \\
Q(s=2,a=0) & Q(s=2,a=1) \\
Q(s=3,a=0) & Q(s=3,a=1) \\
Q(s=4,a=0) & Q(s=4,a=1) \\
\end{bmatrix}
}
\right.
% This is the visible matrix
\overset{\xrightarrow{\text{actions}}}
{
\begin{bmatrix} 
Q(s=0,a=0) & Q(s=0,a=1) \\
Q(s=1,a=0) & Q(s=1,a=1) \\
Q(s=2,a=0) & Q(s=2,a=1) \\
Q(s=3,a=0) & Q(s=3,a=1) \\
Q(s=4,a=0) & Q(s=4,a=1) \\
\end{bmatrix}
}
$$

In [5]:
rewards = np.zeros([num_states, num_actions])
print(rewards)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


Reset the environment. Then take an action (<font color='brown'>0 or 1</font>) and assign the reward to the table. Try a few actions. For now, just <b>observe how the rewards accumulate</b>. Later, you will improve upon this method.

In [12]:
import random
actions = [0, 1]
actions_weights = [0.9, 0.1]

def loop_actions(num=20):
    state = env.reset()
    for _ in range(num):
        action = random.choices(actions, weights=actions_weights, k=1)[0]
        state_next, reward, _, _ = env.step(action)
        rewards[state, action] = reward
        transition = "s=%d, a=%d, r=%d, s'=%d" % (state, action, reward, state_next)
        clear_output(wait = True)
        print(transition)
        print(rewards)
        state = state_next

In [7]:
# Check how random.choices work
random.choices(actions, weights=actions_weights, k=20)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]

In [13]:
loop_actions()

s=2, a=0, r=0, s'=3
[[ 0.  0.]
 [ 0.  2.]
 [ 0.  0.]
 [ 0.  0.]
 [10.  2.]]


Observe that the agent stays in the easy-to-reach states. Explore more state-action pairs by running the agent in a loop.

To run the agent in a loop, you must automatically choose actions. **The algorithm that chooses actions is called the <font color='darkblue'>policy</font>**. A simple policy algorithm is to choose actions randomly. Run the code below to define the **random policy**:

In [9]:
def policy_random(num_actions):
    return np.random.randint(0, num_actions)

Alternatively, sample a random action using the Gym library's built-in API:

In [10]:
def policy_random(env):
    return env.action_space.sample()

Run the following cell to run a full episode of the agent. For each transition tuple $(s,a,r,s')$, the agent assigns the reward `r` to the corresponding table cell `[s,a]`.

Run the cell a few times and observe how the rewards table changes. Additionally, print the episode's total reward, called the **return**. Does the rewards table vary on each episode? Why? Expand the succeeding "Solution" section for the answer.

In [11]:
state = env.reset()
done = False
rewards_table = np.zeros([num_states, num_actions])
episode_return = 0

while not done: # episode terminates after 1000 actions
    action = policy_random(env)
    state_next, reward, done, _ = env.step(action)
    episode_return += reward
    rewards_table[state, action] = reward
    state = state_next
    print(rewards_table)
    clear_output(wait = True)

print(rewards_table)
print("Return: "+str(episode_return))

[[ 0.  2.]
 [ 0.  2.]
 [ 0.  2.]
 [ 0.  2.]
 [10.  2.]]
Return: 1268


<a id='sect2_1'></a>
### <font color='darkgreen'>Solution (expand to view)</font>

**Yes, the rewards table varies on every episode, because the environment is probabilistic**. Specifically, every $(s,a)$ transition has multiple possible outcomes $(r,s')$. The current rewards table only reflects the last recorded outcome because every outcome overwrites the previous outcome.

For example, while (s=0,a=0) usually leads to (r=0, s'=1), sometimes it leads to (r=2, s'=0). Similarly, other state-action pairs that return r=0 with high probability have nonzero rewards. **<font color='darkred'>Therefore, because transitions are probabilistic, an agent cannot rely on a single transition to calculate reward. Instead, the agent must weight reward over multiple transitions**</font>.

<a id='sect3'></a>
## <font color='darkblue'>Learning Probabilistic Rewards</font> ([back](#sect0))

Initialize the approximate reward $R(s,a)$ for each $(s,a)$ pair with some value, say $0$. Then, you can gradually refine $R(s,a)$ to approximate the probabilistic reward by adding a correction on each state transition.

Programmatically, represent a correction to the approximated $R(s,a)$ by using the following update rule:

$$R(s,a) \gets R(s,a) + correction$$

Weight each correction by a learning rate $\alpha$. Now, you can repeat this correction for multiple state transitions. <b>By weighting each correction to the reward, the final approximated reward reflects all the probabilistic state transitions experienced by the agent.</b>

$$R(s,a) \gets R(s,a) + \alpha \cdot correction$$

By definition, a correction is the difference between the measured reward $r_{s,a}$ and the expected reward $R(s,a)$:

$$R(s,a) \gets R(s,a) + \alpha(r_{s,a} - R(s,a))$$

Program this update rule in the following cell where indicated by `TODO`. Then run the code cell to generate the rewards table. On first glance, this rewards table looks promising. Do you think this rewards table could help the agent reach the big reward of 10?

In [14]:
learning_rate = 0.1

state = env.reset()
done = False

rewards_table = np.zeros([num_states, num_actions])
episode_return = 0

while not done: # episode terminates after 1000 actions
    action = policy_random(env)
    state_new, reward, done, _ = env.step(action)
    episode_return += reward
    rewards_table[state, action] += learning_rate * (reward - rewards_table[state,action]) # TODO: Code the update rule
    state = state_new
    print(rewards_table)
    clear_output(wait = True)

print(rewards_table)
print("Return: " + "{:.2f}".format(episode_return))

[[0.36 1.32]
 [0.26 1.69]
 [0.41 1.54]
 [0.16 1.53]
 [8.12 3.39]]
Return: 1348.00


<a id='sect3_1'></a>
### <font color='darkgreen'>Solution (expand to view code)</font>

Run the following cell to implement the update rule and generate the rewards table.

**This rewards table will not help the agent reach the big reward.** To reach the big reward, the agent must repeat a=0 till it reaches state 4. However, the rewards table tells an agent in state 0 that a=1 returns a larger reward than a=0. <font color='darkred'>**The rewards table does not account for the fact that taking a=0 brings the agent closer to the big reward. In other words, for each $(s,a)$ pair, the rewards table only tracks immediate reward instead of capturing the total possible reward.**</font>

In the next section, you will learn how to estimate total reward.

In [15]:
learning_rate = 0.1

state = env.reset()
done = False

rewards_table = np.zeros([num_states, num_actions])
episode_return = 0

while not done: # episode terminates after 1000 actions
    action = policy_random(env)
    state_new, reward, done, _ = env.step(action)
    episode_return += reward
    rewards_table[state,action] += learning_rate * (reward - rewards_table[state, action])
    state = state_new
    print(rewards_table)
    clear_output(wait = True)

print(rewards_table)
print("Return: " + "{:.2f}".format(episode_return))

[[0.27 1.75]
 [0.11 1.84]
 [0.52 1.61]
 [0.45 1.46]
 [8.61 3.68]]
Return: 1440.00


<a id='sect4'></a>
## <font color='darkblue'>Q-Function and Q-Values</font> ([back](#sect0))

<b><font size='3ptx'>Q-learning is the *fundamental* concept in this course. Ensure you understand this section.</font></b>

<b>You must calculate the <font color='darkblue'>return</font>, not the immediate reward</b>. For a given $(s_0,a_0)$, the **return** is  the sum of all rewards until the episode terminates, denoted by $Q(s_0,a_0)$:

$$ Q(s_0,a_0) = r_{s_0,a_0} + r_{s_1,a_1} + \ldots + r_{s_{n-1}, a_{n-1}} $$

Because the environment's rewards are probabilistic, $Q(s_0,a_0)$ is the *expected* return, called the <font color='darkblue'>**Q-function**</font> or the **state-action** function.

In the formula above, $Q(s_0,a_0)$ weights more distant rewards equally with less distant rewards. However, closer rewards are more desirable because they maximize reward faster. Therefore, <b>account for the delayed nature of future rewards by introducing a discount factor<font color='darkblue'> $\gamma$ </font>.</b> 

$$ Q(s_0,a_0) = r_{s_0,a_0} +\gamma r_{s_1,a_1} + \gamma^2 r_{s_2,a_2} + \ldots + \gamma^{n-1} r_{s_{n-1}, a_{n-1}} $$

<br/>
    
Notice that the equation is recursive:

$$ Q(s_0,a_0) = r_{s_0,a_0} + \gamma Q(s_1,a_1) $$

In this equation, you determine action $a_1$ using some policy, such as a random policy. Therefore, $Q(s_0,a_0)$ is the return from taking an action $a$ in a state $s$ and then following some policy that determines the future actions $a_1, a_2, \ldots$.

So far, your agent has chosen the action $a_1$ randomly. However, <b>your agent should choose whatever action maximizes return</b>. Modify the equation to choose the action $a_1$ that maximizes return:

$$Q(s_0,a_0) = r_{s_0,a_0} +  \gamma \displaystyle \max_{\substack{a_1}} Q(s_1,a_1)$$

Using this equation, an agent can update the approximated Q-values by using the update rule from [Learning Probabilistic Rewards](#scrollTo=ljLCwwHmMXnz) as follows:

$$Q_{updated}(s_0,a_0) \gets Q_{old}(s_0,a_0) + \alpha \cdot (Q_{calculated} - Q_{old}(s_0,a_0))$$

Substituting for $Q_{calculated}$ with the expression for $Q(s_0,a_0)$:

$$Q_{upd}(s_0,a_0) \gets Q_{old}(s_0,a_0) + \alpha\underbrace{
  \left(
      \overbrace{
        \underbrace{
        r_{s_0,a_0}
        }_{\text{new reward}}
      + \gamma \displaystyle \max_{\substack{a_1}} Q(s_1,a_1)
      }^{\text{calculated } Q(s_0,a_0)}
    - Q_{old}(s_0,a_0) \right)
    }_{\text{error gradient}}
$$

This equation looks intimidating. Remember that the terms in the square brackets represent the correction, called the <font color='darkblue'>**error gradient**</font>. In the error gradient, on each transition, the only new information is $r(s_0,a_0)$. The agent uses $r(s_0,a_0)$ to calculate the new Q-value $Q(s_0,a_0)$, then subtracts the old Q-value to get the error gradient, and finally weights the error gradient by a learning rate.

<b>The equation to update $Q$ is the famous <a href='https://en.wikipedia.org/wiki/Bellman_equation'>Bellman equation</a> as applied to RL</b>. Take a moment to ensure you understand the equation.


<a id='sect5'></a>
## <font color='darkblue'>Implement Bellman Equation</font> ([back](#sect0))

Define a function that runs an episode and updates the Q-values by completing the missing Bellman update in the following cell. Your implementation must reproduce the last equation in the section above. The argument `learning_rate` is $\alpha$ and the argument `discount_factor` is $\gamma$.

Check your implementation against the solution.

In [28]:
#@title Solution (to view code, from cell's menu, select Form -> Show Code)
def run_training_episode(env, q_table, learning_rate, discount_factor):
    state = env.reset()
    done = False
    while(not done):
        action = policy_random(env)
        state_new, reward, done, _ = env.step(action)
        q_table[state, action] = q_table[state, action] + learning_rate * (
            reward + discount_factor * np.max(q_table[state_new,:]) - \
            q_table[state, action]
        )
        state = state_new
        
    return(q_table)

<a id='sect6'></a>
## <font color='darkblue'>Train the Agent to Solve NChain</font> ([back](#sect0))

To train the agent, define a function that runs multiple episodes. For each episode, the function calls `run_training_episode` and prints the Q-table. This output shows the Q-table evolving over episodes. Fill out the call to `run_training_episode`.

In [23]:
#@title Solution (to view code, from cell's menu, select Form -> Show Code)
def train_agent(env, episodes, learning_rate, discount_factor):
    q_table = np.zeros([num_states, num_actions])
    for episode in range(episodes):
        q_table = run_training_episode(env, q_table, learning_rate, discount_factor)
        print(q_table)
        clear_output(wait = True)
    return(q_table)

Recall that the Q-table stores Q-values for every state-action pair in the following format:

$$
\text{states}\left\downarrow\vphantom{
% the following matrix is invisible
% the \vphantom uses the matrix to know its size
\begin{bmatrix} 
Q(s=0,a=0) & Q(s=0,a=1) \\
Q(s=1,a=0) & Q(s=1,a=1) \\
Q(s=2,a=0) & Q(s=2,a=1) \\
Q(s=3,a=0) & Q(s=3,a=1) \\
Q(s=4,a=0) & Q(s=4,a=1) \\
\end{bmatrix}
}
\right.
% This is the visible matrix
\overset{\xrightarrow{\text{actions}}}
{
\begin{bmatrix} 
Q(s=0,a=0) & Q(s=0,a=1) \\
Q(s=1,a=0) & Q(s=1,a=1) \\
Q(s=2,a=0) & Q(s=2,a=1) \\
Q(s=3,a=0) & Q(s=3,a=1) \\
Q(s=4,a=0) & Q(s=4,a=1) \\
\end{bmatrix}
}
$$

Now, before training your agent, consider these questions with respect to the Q-table:

 * Should $Q(s=0,a=0)$ be higher than $Q(s=0,a=1)$?
 * Should $Q(s=0,a=1)$ be higher than $Q(s=2,a=0)$?
 * How does the answer to these questions depend on $\gamma$?
 
Remember that <font color='darkred'><b>$Q$ measures the return, not the reward</b></font>. Refer to the following graph for the environment when considering those questions:

<img alt="A schematic that shows the NChain environment. The schematic shows the states, possible actions, and results of taking those actions in the state. The result is a new state and a reward. There are 5 states. The allowed actions from each state are labelled 0 and 1. Action 0 always leads to a reward of 0, except from state 4 where action 0 returns a reward of 10. Action 1 always returns a reward of 2." width="50%" src="images/2.PNG"/>

<a id='sect6_1'></a>
### <font color='darkgreen'>Answers (expand to view)</font>

 * Should $Q(s=0,a=0)$ be higher than $Q(s=0,a=1)$?
    *  Yes, because $s=0,a=0$ leads to the big reward, and therefore a higher return. Q-values measure return, not reward.
 * Should $Q(s=0,a=1)$ be higher than $Q(s=2,a=0)$?
   * No, because $(s=0,a=1)$ prevents the agent from reaching the big reward, while $(s=2,a=0)$ brings the agent closer to the big reward. Therefore, $(s=2,a=0)$ leads to a higher return ( Q-value).
 * How does the answer to these questions depend on $\gamma$?
   * $\gamma$ determines the increase in Q-value of a state-action pair from later rewards. <b>A higher $\gamma$ increases the propagation of rewards to Q-values of preceding state-action pairs</b>. Hence, the previous two answers hold for a high $\gamma$ but not for a low $\gamma$.

<a id='sect6_2'></a>
### <font color='darkgreen'>Run Training</font>

Run the following cell to calculate Q-values over multiple episodes. Adjust `learning_rate`, `discount_factor`, and `episodes` to return Q-values that match your expectations. For the solution, expand the following section.

In [31]:
discount_factor = 0.8   #@param
learning_rate = 0.01   #@param
episodes = 10   #@param

q_table = train_agent(env, episodes, learning_rate, discount_factor)
print(q_table)

[[ 6.47  7.79]
 [ 6.39  7.76]
 [ 6.8   7.64]
 [ 9.88  8.16]
 [17.31  9.68]]


<a id='sect6_3'></a>
### <font color='darkgreen'>Solution</font>

Run the following code to solve the environment. The rewards table shows that the best action is always 0. See discussion in the following cell.

In [32]:
discount_factor = 0.95   #@param
learning_rate = 0.1    #@param
episodes = 10   #@param

q_table = train_agent(env, episodes, learning_rate, discount_factor)
print(q_table)

[[50.93 50.49]
 [53.85 51.08]
 [58.28 52.16]
 [63.06 53.63]
 [71.99 56.79]]


The solution above uses `learning_rate = 0.1`. In RL in general, `0.1` is a high value for `learning_rate`. RL agents typically learn environments using a much lower value of `learning_rate`. However, your agent can learn `NChain` using a high value of `learning_rate` because `NChain` is a very simple environment.

In fact, your agent can learn `NChain` in one episode using a learning rate of `0.5`. Try it.

<a id='sect7'></a>
## <font color='darkblue'>Test Your Trained Agent</font> ([back](#sect0))

You've completed the hard work of training your agent. <b>Now let's compare your trained agent to an agent choosing random actions.</b> Your trained agent can maximize reward from every state transition by following <b>a policy that always chooses the action with the highest Q-value. Such a policy is called a <font color='darkblue'>greedy policy</font></b>.

Define a function that runs your agent by following either a random policy or a greedy policy:

In [33]:
def run_episode(env, q_table, policy_flag):
    state = env.reset()
    done = False
    episode_return = 0
    while(not done):
        if(policy_flag=='random'):
            action = env.action_space.sample()
        elif(policy_flag=='greedy'):
            action = np.argmax(q_table[state,:])
        else:
            raise Exception("Error: Policy flag must be 'random' or 'greedy'.")
        state_new, reward, done, _ = env.step(action)
        episode_return += reward
        state = state_new
    return(episode_return)

def run_agent(env, episodes, q_table, policy_flag):
    reward_avg = 0.0
    for episode in range(episodes):
        reward_avg += run_episode(env, q_table, policy_flag)
    return(reward_avg/episodes)

Compare the average reward found by random and greedy agents over 10 episodes.

In [34]:
episodes = 10
print("Average returns over " + str(episodes) + " episodes by -")
print("Trained agent: " + \
     str(run_agent(env, episodes, q_table, 'greedy')))
print("Random agent: " + \
     str(run_agent(env, episodes, q_table, 'random')))

Average returns over 10 episodes by -
Trained agent: 3572.6
Random agent: 1328.6


<b>The trained agent is superior to the random agent.</b>

<a id='sect8'></a>
## <font color='darkblue'>Contrasting RL with Supervised Learning</font> ([back](#sect0))

You might notice that the equations for gradient descent and the Bellman equation look similar. The equation for gradient descent is:
$$a_{n+1} = a_n - \gamma \nabla F(a_n)$$

The difference is in the gradient calculation. In <b><a href='https://en.wikipedia.org/wiki/Supervised_learning'>supervised learning</a></b>, the loss is the gradient of the loss function, which is the delta between the predicted and actual values. In RL, the loss is the gradient of the delta between the newly estimated return and the old estimate of the return.

<a id='sect9'></a>
## <font color='darkblue'>Conclusion and Next Steps</font>  ([back](#sect0))

You learned how to solve a simple environment using Q-learning. In the next Colab, you'll learn how to solve a more complex environment using Q-learning.

Move onto the next Colab: [Tabular Q-Learning](https://colab.research.google.com/drive/1sX2kO_RA1DckhCwX25OqjUVBATmOLgs2#forceEdit=true&sandboxMode=true?utm_source=ss-reinforcement-learning&utm_campaign=colab-external&utm_medium=referral&utm_content=rl-tabular-q-learning).

For reference, the sequence of course Colabs is as follows:

1. [Problem Framing in Reinforcement Learning](https://colab.research.google.com/drive/1sUYro4ZyiHuuKfy6KXFSdWjNlb98ZROd#forceEdit=true&sandboxMode=true?utm_source=ss-reinforcement-learning&utm_campaign=colab-external&utm_medium=referral&utm_content=rl-problem-framing)
1. [Q-learning Framework](https://colab.research.google.com/drive/1ZPsEEu30SH1BUqUSxNsz0xeXL2Aalqfa#forceEdit=true&sandboxMode=true?utm_source=ss-reinforcement-learning&utm_campaign=colab-external&utm_medium=referral&utm_content=rl-q-learning)
1. [Tabular Q-Learning](https://colab.research.google.com/drive/1sX2kO_RA1DckhCwX25OqjUVBATmOLgs2#forceEdit=true&sandboxMode=true?utm_source=ss-reinforcement-learning&utm_campaign=colab-external&utm_medium=referral&utm_content=rl-tabular-q-learning)
1. [Deep Q-Learning](https://colab.research.google.com/drive/1XnFxIE882ptpO83mcAz7Zg8PxijJOsUs#forceEdit=true&sandboxMode=true?utm_source=ss-reinforcement-learning&utm_campaign=colab-external&utm_medium=referral&utm_content=rl-deep-q-learning)
1. [Experience Replay and Target Networks](https://colab.research.google.com/drive/1DEv8FSjMvsgCDPlOGQrUFoJeAf67cFSo#forceEdit=true&sandboxMode=true?utm_source=ss-reinforcement-learning&utm_campaign=colab-external&utm_medium=referral&utm_content=rl-experience-replay-and-target-networks)